In [49]:
#imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset ,DataLoader
from tqdm.notebook import tqdm
import itertools as it

### Input / output format
Input is in de vorm van twee vectors die waardes 1 en 0 bevatten (waar of onwaar).\
De waardes van de **eerste** vector moeten worden gelezen alsof er conjuncten tussen staan, een voorbeeld is de vector (0,1) die kan worden gelezen als: niet A en B.\
De waardes van de **tweede** vector moeten worden gelezen alsof er disjuncten tussen staan, verder zijn er altijd exact twee true values in de vector.\
Deze vectoren worden vergeleken (bijvoorbeeld (0,0,1) en (0,1,1) is True) en de output zal true of false zijn.\
De hoeveelheid mogelijke combinaties zijn 2^n * (n!/(2!(n-2)!)), voor n = 3 is dit 24.\
\
Alle combinaties waar de eerste vector gelijk is aan de tweede vector worden verwijderd.\
\
In deze poging ga ik de tien keer regel gebruiken, dit houd in dat voor elk mogelijk input-output paar er tien kopieëren.

In [50]:
#generate input vectors
def generate_input_vectors(n, repeats=10):
    fist_vec = list(it.product([0, 1], repeat=n))
    second_vec = []
    for i in fist_vec:
        if i.count(1) == 2:
            second_vec.append(i)
    total_vec = list(it.product(fist_vec, second_vec))
    for i in total_vec:
        for j in range(len(i[0])):
            if i[0][j] == 0 and i[1][j] == 1:
                break
            elif j == len(i[0]) - 1: 
                total_vec.remove(i)
    total_vec = total_vec * repeats
    return total_vec
#generate validation vectors
def generate_val(n):
    fist_vec = list(it.product([0, 1], repeat=n))
    second_vec = []
    for i in fist_vec:
        if i.count(1) == 2:
            second_vec.append(i)
    total_vec = list(it.product(fist_vec, second_vec))
    return total_vec
#generate true output
def generate_true(x):
    out = []
    for pair in x:
        for i in range(len(pair[0])):
            if pair[0][i] == 1 and pair[1][i] == 1:
                out.append(((pair[0],pair[1]),True))
                break
            elif i == len(pair[0]) - 1:
                out.append(((pair[0],pair[1]),False))
    return out
test_vec = generate_true(generate_input_vectors(4))
for i in test_vec:
    if i[1] == True and i[0][0].count(1) > 2:
        print(i)

(((0, 1, 1, 1), (0, 1, 0, 1)), True)
(((0, 1, 1, 1), (1, 0, 0, 1)), True)
(((0, 1, 1, 1), (1, 0, 1, 0)), True)
(((0, 1, 1, 1), (1, 1, 0, 0)), True)
(((1, 0, 1, 1), (0, 1, 0, 1)), True)
(((1, 0, 1, 1), (0, 1, 1, 0)), True)
(((1, 0, 1, 1), (1, 0, 1, 0)), True)
(((1, 0, 1, 1), (1, 1, 0, 0)), True)
(((1, 1, 0, 1), (0, 0, 1, 1)), True)
(((1, 1, 0, 1), (0, 1, 1, 0)), True)
(((1, 1, 0, 1), (1, 0, 1, 0)), True)
(((1, 1, 1, 0), (0, 0, 1, 1)), True)
(((1, 1, 1, 0), (0, 1, 0, 1)), True)
(((1, 1, 1, 0), (1, 0, 0, 1)), True)
(((1, 1, 1, 0), (1, 1, 0, 0)), True)
(((1, 1, 1, 1), (0, 1, 0, 1)), True)
(((1, 1, 1, 1), (1, 0, 0, 1)), True)
(((1, 1, 1, 1), (1, 1, 0, 0)), True)
(((0, 1, 1, 1), (0, 1, 0, 1)), True)
(((0, 1, 1, 1), (1, 0, 0, 1)), True)
(((0, 1, 1, 1), (1, 0, 1, 0)), True)
(((0, 1, 1, 1), (1, 1, 0, 0)), True)
(((1, 0, 1, 1), (0, 1, 0, 1)), True)
(((1, 0, 1, 1), (0, 1, 1, 0)), True)
(((1, 0, 1, 1), (1, 0, 1, 0)), True)
(((1, 0, 1, 1), (1, 1, 0, 0)), True)
(((1, 1, 0, 1), (0, 0, 1, 1)), True)
(

In [51]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.X = [torch.tensor(i[0]) for i in data]
        self.y = torch.tensor([i[1] for i in data], dtype=torch.bool)
        self.data = list(zip(self.X, self.y))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [52]:
formula_size = 8

data_train = generate_true(generate_input_vectors(formula_size))
dataset_train = CustomDataset(data_train)
dataloader_train = DataLoader(dataset_train, batch_size=4, shuffle=True)

print(f"Train size = {len(dataset_train)}")

Train size = 59200


In [53]:
def random_formula(size):
    return torch.randint(0, 2, (size,))

In [54]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(2*formula_size, formula_size)
        self.fc2 = nn.Linear(formula_size, 2)

    def forward(self, x):
        x = x.view(-1, 2*formula_size)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = NeuralNet()

In [55]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [56]:
preffered_device = 'cpu'
second_option = 'cuda'
device = torch.device(second_option if torch.cuda.is_available() else preffered_device)
print(device)

cpu


In [57]:
epochs = 10

model.to(device)

for epoch in range(epochs):
    model.train()
    for inputs, labels in dataloader_train:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
    print(f'Epoch: [{epoch+1}/{epochs}], Loss: {loss.item()}')

Epoch: [1/10], Loss: 0.002177207497879863


KeyboardInterrupt: 

In [ ]:
# Generate validation data
data_val = generate_true(generate_val(formula_size))
dataset_val = CustomDataset(data_val)
dataloader_val = DataLoader(dataset_val, batch_size=4, shuffle=True)

In [ ]:
correct = 0
total = 0
wrong_input = []
wrong_label = []
wrong_output = []
# evaluate the trained model
with torch.no_grad():
    model.eval()
    for inputs, labels in dataloader_val:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #store the wrong predictions
        for i in range(len(predicted)):
            if predicted[i] != labels[i]:
                wrong_input.append(inputs[i])
                wrong_label.append(labels[i])
                wrong_output.append(predicted[i])

# evaluate the random model
random_correct = 0
random_total = 0
predictions = random_formula(len(dataset_val))
for i in range(len(dataloader_val)):
    if (predictions[i] == 1 and dataset_val[i][1] == True) or (predictions[i] == 0 and dataset_val[i][1] == False):
        random_correct += 1
        random_total += 1
    else:
        random_total += 1

print(f'Accuracy of trained model: {100 * correct / total}%')
print(f'Accuracy of random model: {100 * random_correct / random_total}%')

Accuracy of trained model: 99.10714285714286%
Accuracy of random model: 51.89732142857143%


### Wrong predictions

In [ ]:
# visualize wrong predictions
for i in range(len(wrong_input)):
    print(f'Input: {wrong_input[i]}, Label: {wrong_label[i]}, Output: {wrong_output[i]}')


Input: tensor([[1, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 1]]), Label: True, Output: 0
Input: tensor([[0, 1, 0, 1, 0, 0, 1, 0],
        [1, 0, 0, 0, 0, 0, 0, 1]]), Label: False, Output: 1
Input: tensor([[1, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 1, 0, 0, 0, 0, 0]]), Label: True, Output: 0
Input: tensor([[1, 1, 0, 1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0, 0, 0, 0]]), Label: True, Output: 0
Input: tensor([[0, 0, 0, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 1]]), Label: True, Output: 0
Input: tensor([[0, 0, 0, 0, 1, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 1]]), Label: True, Output: 0
Input: tensor([[1, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 1]]), Label: True, Output: 0
Input: tensor([[1, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0]]), Label: True, Output: 0
Input: tensor([[0, 0, 0, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 1]]), Label: True, Output: 0
Input: tensor([[1, 0, 0, 1, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 1]]), Label: True, Output: 0